# Data Cleaning Listado de Listado de Beneficiarios 2022

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_22 -> Dataset con el listado de TODOS los beneficiarios de 2022.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)  #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2022

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_22 = pd.read_csv(
    '../../data/productores_beneficiarios 2019-2022/listado_beneficiarios_fertilizantes_2022.csv', encoding='utf-8',
    skiprows=0)

In [4]:
beneficiarios_22 = beneficiarios_22[~(
            (beneficiarios_22['ENTIDAD'] == 'NACIONAL') & (beneficiarios_22['MUNICIPIO'] == 'NACIONAL') & (
                beneficiarios_22['LOCALIDAD'] == 'NACIONAL'))]

In [5]:
beneficiarios_22.shape

(826385, 16)

In [6]:
nan_rows = beneficiarios_22[beneficiarios_22.isna().any(axis=1)]

In [7]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
826382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
826384,"""Nota: El Sistema de Rendición de Cuentas sobr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
beneficiarios_22['ENTIDAD'].unique()

array(['GUERRERO', 'OAXACA', 'PUEBLA', 'MORELOS', 'CHIAPAS', 'DURANGO',
       'TLAXCALA', 'NAYARIT', 'ZACATECAS', nan], dtype=object)

Dropeamos los tres valores nulos debido a que las rows enteras salen como nan

In [9]:
beneficiarios_22.dropna(inplace=True)

In [10]:
# Obtenemos las localidades únicas en el dataset.
Localidades_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_22 = Localidades_22.drop_duplicates()


In [11]:
Localidades_22['ENTIDAD_c_benef'] = Localidades_22['ENTIDAD'].apply(clean_text)
Localidades_22['MUNICIPIO_c_benef'] = Localidades_22['MUNICIPIO'].apply(clean_text)
Localidades_22['LOCALIDAD_c_benef'] = Localidades_22['LOCALIDAD'].apply(clean_text)

In [12]:
# Creamos las dos keys de beneficiarios 2019
Localidades_22['KEY_benef_mun'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_22['KEY_benef_loc'] = Localidades_22['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_22[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_22['LOCALIDAD_c_benef'].astype(str)

In [13]:
Localidades_22.shape

(12513, 8)

In [14]:
# Obtenemos las localidades únicas en el dataset.
Municipios_22 = beneficiarios_22[['ENTIDAD', 'MUNICIPIO']]
Municipios_22 = Municipios_22.drop_duplicates()

In [15]:
# Estandarizamos la limpieza de los datos
Municipios_22['ENTIDAD_c_benef'] = Municipios_22['ENTIDAD'].apply(clean_text)
Municipios_22['MUNICIPIO_c_benef'] = Municipios_22['MUNICIPIO'].apply(clean_text)

In [16]:
# Creamos las dos keys de beneficiarios 
Municipios_22['KEY_benef_mun'] = Municipios_22['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_22[
    'MUNICIPIO_c_benef'].astype(str)

In [17]:
Municipios_22.shape

(1029, 5)

In [18]:
Municipios_22.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2022

Obtener listado único de municipios y listado único de localidades de inegi de 2022 con sus claves.

In [19]:
path_dataset_inegi_2022 = '../../data/inegi/dataset_inegi_clean_2022.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2022 = pd.read_csv(path_dataset_inegi_2022)

### 1.2.1 INEGI 2022 Municipios únicos para cada año.

In [20]:
dataset_inegi_2022['KEY_inegi_municipio'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + \
                                            dataset_inegi_2022[
                                                'Municipio_c_inegi'].astype(str)
dataset_inegi_2022['KEY_inegi_localidad'] = dataset_inegi_2022['Entidad_c_inegi'].astype(str) + '-' + \
                                            dataset_inegi_2022[
                                                'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2022[
                                                'Localidad_c_inegi'].astype(str)

In [21]:
INEGI_UNIQUEMUN_2022 = dataset_inegi_2022.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad", "POB_TOTAL"])

INEGI_UNIQUEMUN_2022 = INEGI_UNIQUEMUN_2022.drop_duplicates()

In [22]:
INEGI_UNIQUEMUN_2022.shape

(2475, 7)

In [23]:
INEGI_UNIQUEMUN_2022.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2022, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_22 y el dataset de INEGI correspondiente.

In [24]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]

    df_benef['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))

    return df_merged

In [25]:
diccionario_MUN_22 = fuzzy_merge_benef2019_2022(Municipios_22, INEGI_UNIQUEMUN_2022, 'KEY_benef_mun',
                                                'KEY_inegi_municipio')
diccionario_MUN_22.shape

(1031, 14)

In [26]:
diccionario_MUN_22.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
                                 'Municipio_c_inegi'], inplace=True)
diccionario_MUN_22.drop_duplicates(subset='KEY_benef_mun', keep='first', inplace=True)

In [27]:
diccionario_MUN_22.shape

(1029, 8)

In [28]:
diccionario_MUN_22.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22.csv',
                          index=False)
diccionario_MUN_22_simple = diccionario_MUN_22.drop(
    columns=['best_match', 'match_score', 'CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'])
# # TODO Es necesario modificar el diccionario manual
# diccionario_MUN_22_simple.to_csv(
#     '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22_simple.csv', index=False)

In [29]:
diccionario_MUN_22_simple = pd.read_csv(
    '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_22_simple.csv')

In [30]:
beneficiarios_22.shape

(826382, 16)

In [31]:
duplicates_specific_column = diccionario_MUN_22_simple[
    diccionario_MUN_22_simple.duplicated(subset=['KEY_benef_mun', 'KEY_inegi_municipio'], keep=False)]
print(duplicates_specific_column)

                 KEY_benef_mun        KEY_inegi_municipio
13    oaxaca-san juan mixtepec   oaxaca-san juan mixtepec
14    oaxaca-san juan mixtepec   oaxaca-san juan mixtepec
339  oaxaca-san pedro mixtepec  oaxaca-san pedro mixtepec
340  oaxaca-san pedro mixtepec  oaxaca-san pedro mixtepec


In [32]:
duplicates_specific_column = diccionario_MUN_22_simple[
    diccionario_MUN_22_simple.duplicated(subset='KEY_inegi_municipio', keep=False)]
print(duplicates_specific_column)

                                      KEY_benef_mun  \
0     oaxaca-heroica ciudad de juchitan de zaragoza   
13                         oaxaca-san juan mixtepec   
14                         oaxaca-san juan mixtepec   
339                       oaxaca-san pedro mixtepec   
340                       oaxaca-san pedro mixtepec   
1016                    oaxaca-juchitan de zaragoza   

              KEY_inegi_municipio  
0     oaxaca-juchitan de zaragoza  
13       oaxaca-san juan mixtepec  
14       oaxaca-san juan mixtepec  
339     oaxaca-san pedro mixtepec  
340     oaxaca-san pedro mixtepec  
1016  oaxaca-juchitan de zaragoza  


In [33]:
diccionario_MUN_22_simple = diccionario_MUN_22_simple.drop_duplicates(subset='KEY_inegi_municipio', keep='first')

In [34]:
diccionario_MUN_22_simple = diccionario_MUN_22_simple.drop_duplicates(subset='KEY_benef_mun', keep='first')

In [35]:
nan_rows = beneficiarios_22[beneficiarios_22.isna().any(axis=1)]

In [36]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_22[beneficiarios_22['ENTIDAD'] == 'NACIONAL']

### MERGE

Armamos por partes el dataset definitivo:
1. Juntamos beneficiarios_19 con el diccionario simple. (listado_beneficiario_parte_I)
2. Al df anterior juntamos las claver provenientes del catálogo de Inegi. (listado_beneficiario_parte_II)

In [37]:
beneficiarios_22.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA'],
      dtype='object')

In [38]:
beneficiarios_22.shape

(826382, 16)

In [39]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join

# Clean listado beneficiarios
beneficiarios_22['ESTADO_Clean'] = beneficiarios_22['ENTIDAD'].apply(clean_text)
beneficiarios_22['MUNICIPIO_Clean'] = beneficiarios_22['MUNICIPIO'].apply(clean_text)

# Create KEY in listado beneficiarios
beneficiarios_22['Estado-mun-KEY'] = beneficiarios_22['ESTADO_Clean'].astype(str) + '-' + beneficiarios_22[
    'MUNICIPIO_Clean'].astype(str)

In [40]:
# Hacer el join de la Parte I
listado_beneficiarios_parte_I = pd.merge(beneficiarios_22, diccionario_MUN_22_simple, left_on="Estado-mun-KEY",
                                         right_on="KEY_benef_mun", how='left', suffixes=('_benef', '_inegi'))

In [41]:
listado_beneficiarios_parte_I.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio'],
      dtype='object')

In [42]:
listado_beneficiarios_parte_I.shape

(826382, 21)

In [43]:
print(INEGI_UNIQUEMUN_2022.shape)
INEGI_UNIQUEMUN_2022.drop_duplicates(subset='KEY_inegi_municipio', keep='first', inplace=True)
print(INEGI_UNIQUEMUN_2022.shape)

(2475, 7)
(2473, 7)


In [44]:
# TODO: REVISAR ESTA PARTE (ARTURO)
listado_beneficiarios_parte_II = pd.merge(listado_beneficiarios_parte_I, INEGI_UNIQUEMUN_2022,
                                          left_on="KEY_inegi_municipio",
                                          right_on="KEY_inegi_municipio", how='left', suffixes=('_benef', '_inegi'))

In [45]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD',
       'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE',
       'PRODUCTO', 'FECHA', 'MONTO FEDERAL', 'APOYO', 'ACTIVIDAD', 'ESLABÓN',
       'CICLO AGRÍCOLA', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY',
       'KEY_benef_mun', 'KEY_inegi_municipio', 'CVE_ENT', 'Entidad_inegi',
       'CVE_MUN', 'Municipio_inegi', 'Entidad_c_inegi', 'Municipio_c_inegi'],
      dtype='object')

In [46]:
listado_beneficiarios_parte_II = listado_beneficiarios_parte_II.drop(
    columns=['ESTADO_Clean', 'MUNICIPIO_Clean', 'Estado-mun-KEY', 'KEY_inegi_municipio', 'Entidad_c_inegi',
             'Municipio_c_inegi', 'ESTRATIFICACIÓN', 'PROGRAMA', 'COMPONENTE', 'SUBCOMPONENTE', 'APOYO', 'ACTIVIDAD',
             'ESLABÓN'])

In [47]:
listado_beneficiarios_parte_II.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [48]:
listado_beneficiarios_parte_II.to_csv('../../data/listados_completos/listado_beneficiarios_2022.csv', index=False)

In [49]:
listado_beneficiarios_parte_II.shape

(826382, 14)

In [50]:
listado_beneficiarios_parte_II.isna().sum()

BENEFICIARIO         0
ZONA                 0
ENTIDAD              0
MUNICIPIO            0
LOCALIDAD            0
PRODUCTO             0
FECHA                0
MONTO FEDERAL        0
CICLO AGRÍCOLA       0
KEY_benef_mun      198
CVE_ENT            198
Entidad_inegi      198
CVE_MUN            198
Municipio_inegi    198
dtype: int64

In [51]:
# Assuming listado_beneficiarios_parte_II is your DataFrame
rows_with_nan = listado_beneficiarios_parte_II[listado_beneficiarios_parte_II.isna().any(axis=1)]

# rows_with_nan now contains only the rows with NaN values in any column

In [52]:
rows_with_nan

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,PRODUCTO,FECHA,MONTO FEDERAL,CICLO AGRÍCOLA,KEY_benef_mun,CVE_ENT,Entidad_inegi,CVE_MUN,Municipio_inegi
59305,BARRIOS ROBLERO ESTEBAN,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
76582,BRAVO PEREZ GUALTERIO DIONICIO,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
133667,CRUS ROBLERO CANDIDO,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-10,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
133668,CRUS ROBLERO JOSE PEDRO,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-10,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
141160,CRUZ LOPEZ ALBERTO,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-12,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796076,VAZQUEZ SOLIS NEIME,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
803973,VELAZQUEZ GONZALEZ LORENZO,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
820122,ZALAZ PEREZ DOMITILA,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-09-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN
825316,ZUNUN ROBLERO ELIZABETH,SUR SURESTE,CHIAPAS,BELISARIO DOMINGUEZ,FLOR DE CHIAPAS,Cultivo de maíz grano blanco,2022-12-05,7491.34,PV2022,NaN,NaN,NaN,NaN,NaN


# 3. Join por entidad (Divide y Vencerás)

In [53]:
listado_beneficiarios_2022 = pd.read_csv('../../data/listados_completos/listado_beneficiarios_2022.csv')

In [54]:
listado_beneficiarios_2022.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun', 'CVE_ENT',
       'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi'],
      dtype='object')

In [55]:
listado_beneficiarios_2022['ENTIDAD_c_benef'] = listado_beneficiarios_2022['ENTIDAD'].apply(clean_text)
listado_beneficiarios_2022['MUNICIPIO_c_benef'] = listado_beneficiarios_2022['MUNICIPIO'].apply(clean_text)
listado_beneficiarios_2022['LOCALIDAD_c_benef'] = listado_beneficiarios_2022['LOCALIDAD'].apply(clean_text)

In [56]:
listado_beneficiarios_2022['KEY_benef_loc'] = listado_beneficiarios_2022['MUNICIPIO_c_benef'].astype(str) + '-' + listado_beneficiarios_2022['LOCALIDAD_c_benef'].astype(str)

In [57]:
listado_beneficiarios_2022.shape

(826382, 18)

In [58]:
listado_beneficiarios_2022 = listado_beneficiarios_2022.drop_duplicates(subset=['KEY_benef_loc'])

In [59]:
listado_beneficiarios_2022.shape

(12509, 18)

In [60]:
listado_beneficiarios_2022.ENTIDAD.unique()

array(['GUERRERO', 'OAXACA', 'PUEBLA', 'MORELOS', 'CHIAPAS', 'DURANGO',
       'TLAXCALA', 'NAYARIT', 'ZACATECAS'], dtype=object)

### 3.2.1 Join de localidades - GUERRERO

Empezamos creando los datasets por entiedad del dataset de beneficiarios.


In [61]:
entities = ['GUERRERO', 'OAXACA', 'PUEBLA', 'MORELOS', 'CHIAPAS', 'DURANGO','TLAXCALA', 'NAYARIT', 'ZACATECAS']

# Dictionary to hold DataFrames
dataframes_dict = {}

for entity in entities:
    variable_name = f"Localidades_22_{entity.upper().replace(' ', '_')}"
    dataframe = listado_beneficiarios_2022[listado_beneficiarios_2022["ENTIDAD"] == entity]
    
    # Check if the DataFrame for this entity already exists in the dictionary
    if variable_name in dataframes_dict:
        # If it exists, concatenate the new DataFrame with the existing one
        dataframes_dict[variable_name] = pd.concat([dataframes_dict[variable_name], dataframe])
    else:
        # If it does not exist, create a new entry in the dictionary
        dataframes_dict[variable_name] = dataframe

# Define each DataFrame as a separate variable
for name, df in dataframes_dict.items():
    globals()[name] = df
    print(f"{name} defined with shape: {df.shape}")

Localidades_22_GUERRERO defined with shape: (4613, 18)
Localidades_22_OAXACA defined with shape: (1793, 18)
Localidades_22_PUEBLA defined with shape: (640, 18)
Localidades_22_MORELOS defined with shape: (260, 18)
Localidades_22_CHIAPAS defined with shape: (3252, 18)
Localidades_22_DURANGO defined with shape: (638, 18)
Localidades_22_TLAXCALA defined with shape: (405, 18)
Localidades_22_NAYARIT defined with shape: (333, 18)
Localidades_22_ZACATECAS defined with shape: (575, 18)


In [62]:
listado_beneficiarios_2022.shape

(12509, 18)

### 3.2.2 Diccionarios de Inegi por entidad.

In [63]:
folder_path = '../../data/productores_beneficiarios 2019-2022/diccionarios_E3/2022'

# Listar todos los archivos en la carpeta
files = os.listdir(folder_path)

# Filtrar solo los archivos CSV
csv_files = [file for file in files if file.endswith('.csv')]

# Leer cada archivo CSV y almacenar en variables separadas
for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    # Crear un nombre de variable dinámicamente eliminando la extensión .csv
    var_name = file_name.replace('.csv', '')
    # Leer el archivo CSV y asignar el DataFrame a la variable
    globals()[var_name] = pd.read_csv(file_path)

# Opcional: Imprimir el nombre de las variables creadas y la cantidad de filas y columnas de cada DataFrame
for file_name in csv_files:
    var_name = file_name.replace('.csv', '')
    df = globals()[var_name]
    print(f'{var_name}: {df.shape[0]} filas, {df.shape[1]} columnas')


INEGI_UNIQUELOC_2022_durango: 9115 filas, 11 columnas
INEGI_UNIQUELOC_2022_oaxaca: 12921 filas, 11 columnas
INEGI_UNIQUELOC_2022_tlaxcala: 1685 filas, 11 columnas
INEGI_UNIQUELOC_2022_puebla: 7724 filas, 11 columnas
INEGI_UNIQUELOC_2022_morelos: 1905 filas, 11 columnas
INEGI_UNIQUELOC_2022_nayarit: 4424 filas, 11 columnas
INEGI_UNIQUELOC_2022_chiapas: 22192 filas, 11 columnas
INEGI_UNIQUELOC_2022_zacatecas: 6457 filas, 11 columnas
INEGI_UNIQUELOC_2022_guerrero: 9214 filas, 11 columnas


In [64]:
INEGI_UNIQUELOC_2022_guerrero.shape

(9214, 11)

In [65]:
Localidades_22_GUERRERO_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_GUERRERO, INEGI_UNIQUELOC_2022_guerrero, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_GUERRERO_parte_I.shape

Localidades_22_PUEBLA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_PUEBLA, INEGI_UNIQUELOC_2022_puebla, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_PUEBLA_parte_I.shape

Localidades_22_MORELOS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_MORELOS, INEGI_UNIQUELOC_2022_morelos, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_MORELOS_parte_I.shape

Localidades_22_TLAXCALA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_TLAXCALA, INEGI_UNIQUELOC_2022_tlaxcala, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_TLAXCALA_parte_I.shape

Localidades_22_DURANGO_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_DURANGO, INEGI_UNIQUELOC_2022_durango, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_DURANGO_parte_I.shape

Localidades_22_OAXACA_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_OAXACA, INEGI_UNIQUELOC_2022_oaxaca, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_OAXACA_parte_I.shape

Localidades_22_NAYARIT_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_NAYARIT, INEGI_UNIQUELOC_2022_nayarit, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_NAYARIT_parte_I.shape

Localidades_22_CHIAPAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_CHIAPAS, INEGI_UNIQUELOC_2022_chiapas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_CHIAPAS_parte_I.shape

Localidades_22_ZACATECAS_parte_I = fuzzy_merge_benef2019_2022(Localidades_22_ZACATECAS, INEGI_UNIQUELOC_2022_zacatecas, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_22_ZACATECAS_parte_I.shape

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26453/107522016.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26453/107522016.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benef['match_score'] = [match[1] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_26453/107522016.py:17: SettingWithCopyWarnin

(575, 31)

In [66]:
# Concatenate datasets vertically
diccionario_LOC_22_simple = pd.concat([
    Localidades_22_GUERRERO_parte_I,
    Localidades_22_PUEBLA_parte_I,
    Localidades_22_MORELOS_parte_I,
    Localidades_22_TLAXCALA_parte_I,
    Localidades_22_DURANGO_parte_I,
    Localidades_22_OAXACA_parte_I,
    Localidades_22_NAYARIT_parte_I,
    Localidades_22_CHIAPAS_parte_I,
    Localidades_22_ZACATECAS_parte_I
], axis=0)

# Check the shape of the concatenated DataFrame
diccionario_LOC_22_simple.shape

(12511, 31)

In [67]:
diccionario_LOC_22_simple.columns

Index(['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'LOCALIDAD_c_benef', 'KEY_benef_loc', 'best_match', 'match_score',
       'CVE_ENT_inegi', 'Entidad_inegi_inegi', 'CVE_MUN_inegi',
       'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'Localidad_c_inegi',
       'KEY_inegi_localidad'],
      dtype='object')

In [69]:
diccionario_LOC_22_simple.drop(columns=['BENEFICIARIO', 'ZONA', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'PRODUCTO',
       'FECHA', 'MONTO FEDERAL', 'CICLO AGRÍCOLA', 'KEY_benef_mun',
       'CVE_ENT_benef', 'Entidad_inegi_benef', 'CVE_MUN_benef',
       'Municipio_inegi_benef', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'LOCALIDAD_c_benef', 'best_match',
       'CVE_ENT_inegi', 'Entidad_inegi_inegi', 'CVE_MUN_inegi',
       'Municipio_inegi_inegi', 'CVE_LOC', 'Localidad_inegi', 'POB_TOTAL',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'Localidad_c_inegi',], inplace=True)

In [70]:
diccionario_LOC_22_simple.columns

Index(['KEY_benef_loc', 'match_score', 'KEY_inegi_localidad'], dtype='object')

In [71]:
diccionario_LOC_22_simple.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_LOC_22_simple.csv', index=False)

In [72]:
diccionario_LOC_22_simple.isna().sum()

KEY_benef_loc          0
match_score            0
KEY_inegi_localidad    0
dtype: int64